In [8]:
__author__ = "Alireza Sadabadi"
__copyright__ = "Copyright (c) 2024 Alireza Sadabadi. All rights reserved."
__credits__ = ["Alireza Sadabadi"]
__license__ = "Apache"
__version__ = "2.0"
__maintainer__ = "Alireza Sadabadi"
__email__ = "alirezasadabady@gmail.com"
__status__ = "Test"
__doc__ = "you can see the tutorials in https://youtube.com/@alirezasadabadi?si=d8o7LK_Ai1Hf68is"
import pandas as pd
df = pd.read_csv("EURUSDH4.csv")
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
9995,2025-04-09 20:00:00,1.10199,1.10626,1.09130,1.09453,30874,8,0
9996,2025-04-10 00:00:00,1.09449,1.09860,1.09425,1.09858,14854,8,0
9997,2025-04-10 04:00:00,1.09857,1.09887,1.09608,1.09821,17355,8,0
9998,2025-04-10 08:00:00,1.09822,1.10474,1.09624,1.10369,25385,8,0
9999,2025-04-10 12:00:00,1.10370,1.11301,1.10330,1.11271,23041,8,0


In [9]:
import numpy as np
import pandas_ta as pdta
df['ATR'] = pdta.atr(df['high'],df['low'],df['close'],length=20)
df['RSI'] = pdta.rsi(df['close'])
df['MA40'] = pdta.sma(df['close'],length=40)
df['MA80'] = pdta.sma(df['close'],length=80)
df['MA160'] = pdta.sma(df['close'],length=160)
df['Average'] = pdta.midprice(df['high'],df['low'])


In [10]:
from scipy.stats import linregress
def GetSlope(array):
    y = np.array(array)
    x = np.arange(len(y))
    response = linregress(x,y)
    return response.slope

In [11]:
    backRolling = 7
    
    df['ATRSlope'] = df['ATR'].rolling(window=backRolling).apply(GetSlope, raw=True)
    df['RSISlope'] = df['RSI'].rolling(window=backRolling).apply(GetSlope, raw=True)
    df['MA40Slope'] = df['MA40'].rolling(window=backRolling).apply(GetSlope, raw=True)
    df['MA80Slope'] = df['MA80'].rolling(window=backRolling).apply(GetSlope, raw=True)
    df['MA160Slope'] = df['MA160'].rolling(window=backRolling).apply(GetSlope, raw=True)
    df['AverageSlope'] = df['Average'].rolling(window=backRolling).apply(GetSlope, raw=True)

In [12]:
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume,ATR,RSI,MA40,MA80,MA160,Average,ATRSlope,RSISlope,MA40Slope,MA80Slope,MA160Slope,AverageSlope
9995,2025-04-09 20:00:00,1.10199,1.10626,1.09130,1.09453,30874,8,0,0.006800,47.415184,1.094368,1.087225,1.085732,1.099715,0.000113,-0.814993,0.000543,0.000242,0.000347,0.001405
9996,2025-04-10 00:00:00,1.09449,1.09860,1.09425,1.09858,14854,8,0,0.006677,51.539877,1.094790,1.087429,1.086029,1.098780,0.000121,-2.163259,0.000511,0.000233,0.000333,0.000142
9997,2025-04-10 04:00:00,1.09857,1.09887,1.09608,1.09821,17355,8,0,0.006483,51.145177,1.095279,1.087640,1.086306,1.096560,0.000092,-2.476780,0.000471,0.000220,0.000317,-0.001012
9998,2025-04-10 08:00:00,1.09822,1.10474,1.09624,1.10369,25385,8,0,0.006584,56.463132,1.095873,1.087904,1.086615,1.100410,0.000059,-1.180560,0.000463,0.000216,0.000307,-0.001221
9999,2025-04-10 12:00:00,1.10370,1.11301,1.10330,1.11271,23041,8,0,0.006740,63.504910,1.096712,1.088277,1.086929,1.104625,0.000046,0.275439,0.000497,0.000227,0.000299,-0.000448


In [13]:
tp = 100*1e-4
sl = tp/2 

def Target(numberOfNextCandles, df):
    open = list(df['open'])
    high = list(df['high'])
    low = list(df['low'])
    dfLength = len(df)
    trend = [None] * dfLength
    
    # cc = current candle
    for cc in range (0,dfLength-numberOfNextCandles-2):
        # VOL = value open low
        # VOH = value open high
        VOL = 0
        VOH = 0
        for i in range(1,numberOfNextCandles+2):
            #o_l = open - low
            #o_h = open - high
            o_l = open[cc+1]-low[cc+i]
            o_h = open[cc+1]-high[cc+i]
            VOL = max(o_l, VOL)
            VOH = min(o_h, VOH)

            if ((-VOL <= -tp) and (-VOH < sl)):
                trend[cc] = 1 # downtrend
                break
            if ((VOL > -sl) and (-VOH >= tp)):
                trend[cc] = 2 # uptrend
                break
            else:
                trend[cc] = 0 # range
            
    return trend

In [14]:
df['Target'] = Target(15, df)
df.value_counts('Target')

Target
0.0    6530
2.0    1910
1.0    1543
Name: count, dtype: int64